![alt text](./img/header.png)

# Saltwater Intrusion Exercise

A growing coastal city is interested in developing a wellfield in order to provide potable drinking water to its residents.  You have been contracted by the city to perform a numerical analysis to determine the effects of groundwater withdrawals on the position of the saltwater interface.  The city is particularly interested in knowing if the planned withdrawal rate will result in pumped water that exceeds the drinking water standard for chloride, which is 250 mg/L.  In addition, you have been asked by the city to delineate a wellfield protection zone by determining the approximate recharge area for the wellfield.

![alt text](./img/exB_fig1.png)

The study area consists of a 10 by 10 km area of a coastal plain aquifer system centered on the proposed wellfield (Figure 1).  The hydrogeology of the area consists of a shallow and deep aquifer, and both aquifers intersect the sea floor (Figure 2).  The two aquifers are separated by a discontinuous confining layer that thickens to the east.  The confining layer is present in the eastern part of the study area, but is absent in the western part.  A low permeability unit, which slopes downward to the east, underlies the deep aquifer and can be considered a no flow boundary.  Extensive field investigations have been performed, and the aquifer properties for the different units are summarized in the table below.  Annual average recharge was determined to be 25.4 cm/yr (10 in/yr).

![alt text](./img/exB_fig2.png)

Unfortunately, due to limitations in available property, the city has only one option for wellfield location, which is shown on the map in Figure 1.  The city is hoping to capture at least 25% of the recharge for the area, but there is an obvious concern that excessive pumping from the deep aquifer could cause the saltwater interface to move inland and contaminate the wellfield.

  Unit            | $K_h$ |  $K_v$      |  $S$        | $S_y$ | $n$   | $\alpha_l$ | $\alpha_v$ 
  --------------- | ----- | ----------- | ----------- | ----- | ----- | ---------- | ---------- 
  Shallow aquifer | 100   | 1           | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          
  Confining unit  | 0.001 | 0.001       | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          
  Deep aquifer    | 2000  | 200         | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          



In [ ]:
# Setup the python environment
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
import config

## Exercise B -- Design, run, and calibrate a 2D cross-section model to obtain the steady-state pre-withdrawal distribution of head and salinity

Based on the geometry of the system, the pre-withdrawal conditions can be obtained by running a 2D cross-section model.  The resulting heads and salinities can then be used as initial conditions for the 3D model to evaluate interface movement in response to pumping.  Because 3D saltwater intrusion models can take a long time, across-sectional model is developed first using one row, 100 columns, and 25 model layers. The model has already been constructed for you.

This exercise was designed such that a 2D model could be used to represent pre-withdrawal conditions.  If the hydrogeology were to vary in the north-south direction, or if an irregular boundary were to exist, the problem of obtaining equilibrium conditions would be more difficult.

### Part B1 -- Determine length of simulation period required to reach equilibrium

In [ ]:
# Data path
datapth = os.path.join('data', 'exSEAWAT_B')

# Grid information
nlay = 23
nrow = 1
ncol = 100
delr = 100.
delc = 100.
top = 2.5
botm = np.linspace(-2.5, -112.5, nlay)

# Temporal discretization
nper = 1
nstp = 1
perlen = 100000.

# Ibound
fname = os.path.join(datapth, 'ibound.txt')
ibound = np.loadtxt(fname).reshape((nlay, nrow, ncol))

# Hydraulic properties
fname = os.path.join(datapth, 'hk.txt')
hk = np.loadtxt(fname).reshape((nlay, nrow, ncol))
fname = os.path.join(datapth, 'vk.txt')
vk = np.loadtxt(fname).reshape((nlay, nrow, ncol))
ss = 1.e-5
sy = 0.2

# Recharge
rech = 25.4 / 100. / 365.  # convert cm/yr to m/day

# Transport
dt0 = 0.
sconc = np.zeros((nlay, nrow, ncol), dtype=np.float32)
sconc[(ibound < 0)] = 35.
icbund = np.abs(ibound)
ssm_data = {}
itype = flopy.mt3d.Mt3dSsm.itype_dict()
layers, rows, columns = np.where(ibound < 0)
ssm_per1 = []
for k, i, j in zip(layers, rows, columns):
    ssm_per1.append((k, i, j, 35., itype['CHD']))
ssm_data[0] = ssm_per1

In [ ]:
# Build the flopy SEAWAT model
model_ws = os.path.join('work', 'exSEAWAT_B')
if not os.path.isdir(model_ws):
    os.mkdir(model_ws)
    
modelname = 'b1'
m = flopy.seawat.Seawat(modelname, model_ws=model_ws, exe_name=config.swexe)
dis = flopy.modflow.ModflowDis(m, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, 
                               top=top, botm=botm, perlen=perlen, nstp=nstp)
bas = flopy.modflow.ModflowBas(m, ibound, strt=0.)
lpf = flopy.modflow.ModflowLpf(m, laytyp=1, hk=hk, vka=vk, ss=ss, sy=sy)
rch = flopy.modflow.ModflowRch(m, rech=rech)
oc = flopy.modflow.ModflowOc(m)
pcg = flopy.modflow.ModflowPcg(m, hclose=1.e-3, rclose=1e4)
vdf = flopy.seawat.SeawatVdf(m, mtdnconc=1, mfnadvfd=1, nswtcpl=0, iwtable=0, 
                             densemin=0., densemax=0., denseslp=25./35., denseref=1000.)
btn = flopy.mt3d.Mt3dBtn(m, nlay, nrow, ncol, nper, laycon=lpf.laytyp, htop=top, 
                        dz=dis.thickness.get_value(), prsity=0.2, icbund=icbund,
                        sconc=sconc, nprs=-10)
adv = flopy.mt3d.Mt3dAdv(m, mixelm=-1, percel=0.5)
dsp = flopy.mt3d.Mt3dDsp(m, al=10., trpt=0.1, trpv=0.1, dmcoef=0.)
ssm = flopy.mt3d.Mt3dSsm(m, crch=0, stress_period_data=ssm_data)
gcg = flopy.mt3d.Mt3dGcg(m, isolve=2, cclose=1.e-6)

In [ ]:
# Make plot of the grid
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()

In [ ]:
# Make color flood plot of hydraulic conductivity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
hkpatchcollection = mm.plot_array(np.log(m.lpf.hk.array), cmap='viridis')
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()
cb = plt.colorbar(hkpatchcollection)

In [ ]:
# write the input files
m.write_input()

In [ ]:
# run the model
success, buff = m.run_model(silent=True)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'MT3D001.UCN')
ucnobj = flopy.utils.binaryfile.UcnFile(fname)
times = ucnobj.get_times()
conc = ucnobj.get_data(totim=times[-1])
conc[np.where(ibound != 1)] = np.nan

In [ ]:
# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
#linecollection = mm.plot_grid()
#patchcollection = mm.plot_ibound()
cb = plt.colorbar(cpatchcollection)

In [ ]:
# Load the mas file and make a plot of total mass in aquifer versus time
fname = os.path.join(model_ws, 'MT3D001.MAS')
mas = flopy.mt3d.Mt3dms.load_mas(fname)
f = plt.figure()
ax = f.add_subplot(1, 1, 1)
lines = ax.plot(mas.time, mas.total_mass)

### Animate Simulating using Model Viewer
Use the Modelviewer program to animate salinity as a function of time.  Make sure to use the MT3DMS option in Modelviewer when opening the files. Open Modelviewer and go to File>new.  You can look at a MODFLOW or MT3DMS results. To view the salinity distribution select MT3DMS. For “cnf” scroll to the directory you are running your model in and select the MT3D.cnf file. Then specify the concentration file (MT3D001.UCN), and select “OK”. We will leave the other two boxes blank. .You can leave the time to visualize as it is, or go to another time.  The data type “concentration” will be used in this visualization.  Click “OK”.  A blank box will appear.  Go to Show>solid and you will see some colors appear; however, this view still looks odd.  Why do you think that is?  Could it have something to do with the dimensions of the model?  How do you think this can be fixed? (Hint: go to Toolbox>Geometry and increase the vertical discretization in “z”) You can move the view around with the left-click on the mouse. There is also a place in the menu to animate the concentrations.  

### B1 Questions
 
1. Based on the plot above, has equilibrium been reached?
2. Determine the length of time it takes to reach equilibrium by rerunning the simulation using larger values for pertim.

### Part B2 -- Evaluate the level of numerical dispersion

In B1, the TVD solver was used with a specified Courant number of 0.5, and because the longitudinal and transverse dispersivities ($\alpha_L$ and $\alpha_T$) were specified as 10 and 1 m, respectively, the simulation included dispersion.  In addition to the intended dispersion, the simulation also had some numerical dispersion.  The purpose of this analysis is to determine the approximate level of numerical dispersion by rerunning the previous simulation with the dispersivity values set to zero.  The following code blocks will setup and run this new simulation.

In [ ]:
# This code block will make a new model called b2 and then run it
m.name = 'b2'
m.dsp.al = 0.
m.dsp.trpt = 0.
m.dsp.trpv = 0.
m.write_input()
m.run_model(silent=True)

### B2 Questions
 
1. Plot the results from this simulation.  How do they compare to B1?
2. Has this simulation reached equilibrium?
3. Use Model Viewer to watch an animation of this simulation.

### Part B3 -- Evaluate transport solution schemes and parameters to determine compromise between accuracy and efficiency

The simulation in B1 was performed using TVD with a specified Courant number of 0.5.  Although the B1 simulation took a while to run, the results can be considered accurate.  The purpose of this analysis is to change some of the simulation parameters, and then compare the results with simulation B1.

In [ ]:
# Reset simulation to B1, and then change percel from original value (0.5) to 1.00
m.name = 'b3a'
m.dsp.al = 10.
m.dsp.trpt = 0.1
m.dsp.trpv = 0.1
m.adv.percel = 1.0
m.write_input()
m.run_model(silent=True)

In [ ]:
# Plot the results if you like by copying the appropriate blocks from above.

In [ ]:
# Try running with implicit finite difference scheme
m.name = 'b3b'
m.adv.mixelm = 0  # implicit finite difference
m.adv.nadvfd = 0  # upstream weighting
m.btn.dt0 = 25.
m.write_input()
m.run_model(silent=True)

In [ ]:
# Plot the results if you like by copying the appropriate blocks from above.

### B3 Questions
 
1. Plot the results from this simulation.  How do they compare to B1?  Is this an acceptable compromise between accuracy and efficiency?
2. Has this simulation reached equilibrium?
3. If time permits, try experimenting with different solver parameters.  How fast can you get the simulation to run?

### Part B4 -- Evaluate transport solution schemes and parameters to determine compromise between accuracy and efficiency

After the cross-sectional model was developed, data was "discovered" from 1998 in boxes from a warehouse that belonged to the city.  Data exist from 7 monitoring wells in the area.  The data included water levels and salinities from 1998.

In [ ]:
# This is the observation data that can be used to calibrate the model
#        nm   col  lay head  sal
obs = [('p1', 74, 12, 0.77, 19.),
       ('p2', 74, 17, 2.17, 1.74),
       ('p3', 55, 4, 2.12, 0),
       ('p4', 93, 22, 0.00, 35),
       ('p5', 48, 19, 2.43, 0),
       ('p6', 61, 13, 1.98, 0),
       ('p7', 27, 5, 3.17, 0)]
obs = np.array(obs, dtype=[('name', object), ('column', int), ('layer', int), 
                           ('head', float), ('conc', float)])
obs = obs.view(np.recarray)

In [ ]:
# Your job is to calibrate these factors to change hydraulic conductivity zone values
hk_shallow = 100 * 1.0
hk_conf = 0.001
hk_deep = 2000 * 1.0
vk_shallow = 1.0 * 1.0
vk_conf = 0.001
vk_deep = 200 * 0.5 1.0

# This will setup the hk2 and vk2 and make the substitutions
kzone = np.ones((nlay, nrow, ncol), dtype=np.int)
idx_shallow = np.where(np.float32(hk) == 100)
idx_conf = np.where(np.float32(hk) == 0.001)
idx_deep = np.where(np.float32(hk) == 2000)
hk2 = np.ones((nlay, nrow, ncol), dtype=np.float32) * -999
vk2 = np.ones((nlay, nrow, ncol), dtype=np.float32) * -999
hk2[idx_shallow] = hk_shallow
hk2[idx_conf] = hk_conf
hk2[idx_deep] = hk_deep
vk2[idx_shallow] = vk_shallow
vk2[idx_conf] = vk_conf
vk2[idx_deep] = vk_deep

In [ ]:
# Use run from exercise b3; this will run the new model
m.name = 'b4'
m.adv.mixelm = 0
m.adv.nadvfd = 0
m.btn.dt0 = 25.
m.lpf.hk = hk2
m.lpf.vka = vk2
m.write_input()
m.run_model(silent=True)

In [ ]:
fname = os.path.join(model_ws, m.name + '.hds')
hdobj = flopy.utils.binaryfile.HeadFile(fname)
times = hdobj.get_times()
head = hdobj.get_data(totim=times[-1])
hdobj.close()

fname = os.path.join(model_ws, 'MT3D001.UCN')
cnobj = flopy.utils.binaryfile.UcnFile(fname)
times = cnobj.get_times()
conc = cnobj.get_data(totim=times[-1])
cnobj.close()

simhead = []
simconc = []
for name, column, layer, obshead, obsconc in obs:
    simhead.append(head[layer - 1, 0, column - 1])
    simconc.append(conc[layer - 1, 0, column - 1])
    
fig = plt.figure(figsize=(15, 15))
ax1 = fig.add_subplot(1, 2, 1, aspect='equal')
ax1.scatter(obs.head, simhead, s=60, facecolors='none')
ax1.set_xlim(0, 3.5)
ax1.set_ylim(0, 3.5)
ax1.plot([0, 3.5], [0, 3.5], 'k--')
ax1.set_title('HEAD')
ax1.set_xlabel('OBSERVED')
ax1.set_ylabel('SIMULATED')
ax2 = fig.add_subplot(1, 2, 2, aspect='equal')
ax2.scatter(obs.conc, simconc, s=60, facecolors='none')
ax2.set_xlim(0, 35)
ax2.set_ylim(0, 35)
ax2.plot([0, 35], [0, 35], 'k--')
ax2.set_title('CONCENTRATION')
ax2.set_xlabel('OBSERVED')
ax2.set_ylabel('SIMULATED')


In [ ]:
# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
cb = plt.colorbar(cpatchcollection)